In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import count, sum, avg, max, min


# 1️⃣ Read Silver tables
customers_df = spark.table("silver_db.customers")
orders_df = spark.table("silver_db.orders")

# 2️⃣ Join Silver tables to create staged Gold data
staged_df = orders_df.join(
    customers_df,
    on="customer_id",
    how="inner"
)

# Compute per-customer metrics
staged_agg_df = (
    staged_df.groupBy(
        "customer_id",
        "name",
        "email",
        "city"
    )
    .agg(
        count("order_id").alias("total_orders"),
        sum("amount").alias("total_amount"),
        avg("amount").alias("avg_amount"),
        max("amount").alias("max_order_amount"),
        min("amount").alias("min_order_amount")
    )
    .withColumn("etl_ts", current_timestamp())  # ETL timestamp for tracking updates
)

# 3️⃣ Check if Gold table exists
if not spark.catalog.tableExists("gold_db.customer_analytics"):
    # First run: create the Gold table
    staged_agg_df.withColumnRenamed("etl_ts", "modified_ts") \
        .withColumn("created_ts", current_timestamp()) \
        .write.format("delta") \
        .mode("overwrite") \
        .saveAsTable("gold_db.customer_analytics")

else:
    # 4️⃣ MERGE staged data into existing Gold table
    target = DeltaTable.forName(spark, "gold_db.customer_analytics")

    target.alias("t").merge(
        staged_agg_df.alias("s"),
        "t.customer_id = s.customer_id"
    ) \
    .whenMatchedUpdate(
        condition="""
            NOT (
                t.total_orders <=> s.total_orders AND
                t.total_amount <=> s.total_amount AND
                t.avg_amount <=> s.avg_amount AND
                t.max_order_amount <=> s.max_order_amount AND
                t.min_order_amount <=> s.min_order_amount
            )
        """,
        set={
            "total_orders": "s.total_orders",
            "total_amount": "s.total_amount",
            "avg_amount": "s.avg_amount",
            "max_order_amount": "s.max_order_amount",
            "min_order_amount": "s.min_order_amount",
            "modified_ts": "s.etl_ts"
        }
    ) \
    .whenNotMatchedInsert(
        values={
            "customer_id": "s.customer_id",
            "name": "s.name",
            "email": "s.email",
            "city": "s.city",
            "total_orders": "s.total_orders",
            "total_amount": "s.total_amount",
            "avg_amount": "s.avg_amount",
            "max_order_amount": "s.max_order_amount",
            "min_order_amount": "s.min_order_amount",
            "created_ts": "s.etl_ts",
            "modified_ts": "s.etl_ts"
        }
    ) \
    .execute()

# 5️⃣ Optional: Display Gold table
spark.table("gold_db.customer_analytics").show(truncate=False)
